# Get the data
Same as in EDA notbook

In [33]:
import pandas as pd
import folium
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import re
import seaborn as sns
sns.set_style('white')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [34]:
df = pd.read_csv("../data/data_clean.csv", 
                          encoding='utf_8', 
                          dtype = 'unicode',
                          parse_dates = True,
                          infer_datetime_format = True,
                          low_memory=False)
df = df.drop("Unnamed: 0", axis = 1)
df = df.drop("Restaurants", axis = 1)#True for every row

In [35]:
for column in df.columns :
    if df[column][0] == 'True' or df[column][0] =='False':
    #This tests if the column is a boolean by using the first row for efficiency    
        df[column] = df[column]=='True'
        
    #for some columns we have NaN, in that case we test if we find a True or False value
    elif "True" in df[column].values :
        df[column] = df[column]=='True'
    elif "False" in df[column].values :
        df[column] = df[column]=='True'

In [36]:
cuisine_type = ["American (New)","American (Traditional)","Arts & Entertainment","Asian Fusion","Bakeries","Barbeque","Bars",
"Beer","Breakfast & Brunch","Buffets","Burgers","Cafes","Canadian (New)","Caribbean","Caterers","Chicken Wings",
"Chinese","Cocktail Bars","Coffee & Tea","Comfort Food","Delis","Desserts","Diners","Ethnic Food",
"Event Planning & Services","Fast Food","Food","Food Delivery Services","French","Gastropubs","Gluten-Free",
"Greek","Grocery","Halal","Hot Dogs","Ice Cream & Frozen Yogurt","Indian","Italian","Japanese","Juice Bars & Smoothies",
"Korean","Latin American","Lounges","Mediterranean","Mexican","Middle Eastern","Nightlife","Pizza","Pubs",
"Salad","Sandwiches","Seafood","Soup","Specialty Food","Sports Bars","Steakhouses","Sushi Bars","Tex-Mex",
"Thai","Vegan","Vegetarian","Vietnamese","Wine & Spirits","Wine Bars"]

ambiance = ["romantic","intimate","classy","hipster","divey","touristy","trendy","upscale","casual"]

In [37]:
for column in df[cuisine_type] :
    df[column] = df[column]=="1"

In [38]:
df['stars']=df['stars'].astype(float)
df.Price = pd.to_numeric(df.Price, errors='coerce')
df = df[np.isfinite(df['Price'])]

In [47]:
df.columns

Index(['address', 'business_id', 'city', 'is_open', 'latitude', 'longitude',
       'name', 'postal_code', 'review_count', 'stars',
       ...
       'romantic', 'intimate', 'classy', 'hipster', 'divey', 'touristy',
       'trendy', 'upscale', 'casual', 'name_length'],
      dtype='object', length=128)

# Base rate
The base rate is the size of the most common class divided by the size of the dataset.
Our accuracy should be better than the default rate

In [39]:
print("The most common class for the ratings is ", df["stars"].mode())

baseRate = df[df["stars"] == 4].count()["stars"] / df["stars"].count()
baseRate

The most common class for the ratings is  0    4.0
dtype: float64


0.25211234548583944

# Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [52]:
model_features = cuisine_type + ["Price", "name_length", 'latitude', 'longitude'] + ambiance

X = df[model_features]
y = df["stars"]


In [53]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y) #we label encode the star ratings

In [54]:
# split train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded, test_size=0.2, random_state=72)

In [ ]:
LR = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=5000, multi_class = "auto")
LR.fit(X_train, y_train)

In [ ]:
# train accuracy
#lbfgs: 0.29628
#newton-cg: 0.2962
LR.score(X_train,y_train)

In [ ]:
# test accuracy
#lbfgs: 0.29015
#newton-cg: 29015
LR.score(X_test, y_test)

Our test accuracy is above the baserate but it isn't really a good result

In [31]:
df = df[np.isfinite(df['Price'])]

In [87]:
np.any(np.isnan(df["Price"]))

False

In [85]:
np.all(np.isfinite(df["Price"]))

False

In [98]:
X = df[["name_length","Price"]]
y = df["stars"]

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y) #we label encode the star ratings

# split train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded, test_size=0.2, random_state=72)

In [100]:
LR = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=500, multi_class = "auto")
LR.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=500, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [101]:
print("Train accuracy :",LR.score(X_train,y_train))
print("Test accuracy :",LR.score(X_test, y_test))

Train accuracy : 0.25722458559483646
Test accuracy : 0.263348327791903
